In [3]:
import pandas as pd
import numpy as np
import csv
import requests
import urllib.request, json 
from pprint import pprint
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
#Convenience function for transfermarkt data
def strToDataframe(x):
    x = x.strip()
    firstWord = x[0:x.find(' ')]
    b = x[int(x.find('€'))+1:]
    b = b[int(b.find('€'))+1:]
    b = b[:b.find('mil')]
    c = x[x.find(',') - 2: x.find(',')+2]
    b = b.replace(",", ".")
    c = c.replace(",", ".")
    d = {'Team': [firstWord.upper()], 'TeamWorth': [b], 'TeamAge': [c]}
    stored = pd.DataFrame(data=d)
    return stored

In [9]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.transfermarkt.com.tr/super-lig/startseite/wettbewerb/TR1/plus/1?saison_id=2016"
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'lxml')
odd = pageSoup.find_all("tr", {"class": "odd"})
even = pageSoup.find_all("tr", {"class": "even"})
#Let's look at the first name in the Players list.
EvenDat = {'Team': [], 'TeamWorth': [], 'TeamAge': []}
EvenData = pd.DataFrame(data=EvenDat)
OddDat = {'Team': [], 'TeamWorth': [], 'TeamAge': []}
OddData = pd.DataFrame(data=OddDat)
FullDat = {'Team': [], 'TeamWorth': [], 'TeamAge': []}
FullData = pd.DataFrame(data=FullDat)
for Even in even:
    EvenData = pd.concat([EvenData, strToDataframe(Even.text)])
for Odd in odd:
    OddData = pd.concat([OddData, strToDataframe(Odd.text)])
FullData = pd.concat([OddData, EvenData])
FullData.TeamAge = pd.to_numeric(FullData.TeamAge, errors='coerce')
FullData.TeamWorth = pd.to_numeric(FullData.TeamWorth, errors='coerce')
FullData = FullData.dropna()
FullData.index = FullData['Team']
FullData['Support'] = [30448, 21351, 3208, 2501, 5524, 4236, 4541, 4422, 15723, 16485, 17252, 3792, 16106, 12645, 1416, 2625, 3369, 4594]
FullData

,Team,TeamAge,TeamWorth,Support
Team,,,,
BEŞIKTAŞ,BEŞIKTAŞ,25.3,177.40,30448
GALATASARAY,GALATASARAY,25.1,129.35,21351
BAŞAKŞEHIR,BAŞAKŞEHIR,26.4,72.95,3208
AKHISAR,AKHISAR,26.7,50.20,2501
KAYSERISPOR,KAYSERISPOR,25.0,45.85,5524
GAZIANTEPSPOR,GAZIANTEPSPOR,24.6,40.55,4236
ALANYASPOR,ALANYASPOR,25.5,38.68,4541
KARDEMIR,KARDEMIR,25.8,36.08,4422
ATIKER,ATIKER,25.3,28.75,15723


In [10]:
data = []
with urllib.request.urlopen('https://raw.githubusercontent.com/jokecamp/FootballData/master/Turkey/SuperLig/2016-2017/superlig.json') as url:
    data = json.loads(url.read().decode())
#d = {'week': [data[0]['week']], 'home': [data[0]['matches'][0]['homeTeam']['name']], 'away': [data[0]['matches'][0]['awayTeam']['name']], 'score': [data[0]['matches'][0]['match']['score']] }
d = {'week': [], 'home': [], 'away': [], 'score': [], 'winner': [], 'Difference': [], 'Season': []}
df = pd.DataFrame(data=d)
print(df)
for x in data:
    for y in x['matches']:
        result = 'draw'
        
        if (int([y['match']['score']][0][0]) > int([y['match']['score']][0][-1])):
            result = 'home'
        elif (int([y['match']['score']][0][0]) < int([y['match']['score']][0][-1])):
            result = 'away'
        tem = {'week': [int(x['week'][0:x['week'].index('.')])], 'home': [y['homeTeam']['name']], 'away': [y['awayTeam']['name']], 'score': [y['match']['score']], 'winner': [result], 'Difference': [(int([y['match']['score']][0][0]) - int([y['match']['score']][0][-1]))], 'Season': [2016]}
        temp = pd.DataFrame(data=tem)
        df = pd.concat([df, temp])      
ChgOfI = lambda x: x.replace('İ', 'I')
df['home'] = df['home'].apply(ChgOfI)
df['away'] = df['away'].apply(ChgOfI)
df

Empty DataFrame
Columns: [Difference, Season, away, home, score, week, winner]
Index: []


,Difference,Season,away,home,score,week,winner
0,-1.0,2016.0,BURSASPOR,ADANASPOR A.Ş.,1 - 2,1.0,away
0,0.0,2016.0,ATIKER KONYASPOR,ÇAYKUR RIZESPOR A.Ş.,1 - 1,1.0,draw
0,2.0,2016.0,KASIMPAŞA A.Ş.,TRABZONSPOR A.Ş.,2 - 0,1.0,home
0,3.0,2016.0,ALANYASPOR,BEŞIKTAŞ A.Ş.,4 - 1,1.0,home
0,-2.0,2016.0,AKHISAR BELEDIYE GENÇLIK VE SPOR,KAYSERISPOR,0 - 2,1.0,away
0,2.0,2016.0,GAZIANTEPSPOR,GENÇLERBIRLIĞI,2 - 0,1.0,home
0,1.0,2016.0,FENERBAHÇE A.Ş.,MEDIPOL BAŞAKŞEHIR FK,1 - 0,1.0,home
0,0.0,2016.0,OSMANLISPOR FUTBOL KULÜBÜ,ANTALYASPOR A.Ş.,0 - 0,1.0,draw
0,1.0,2016.0,KARDEMIR KARABÜKSPOR,GALATASARAY A.Ş.,1 - 0,1.0,home
0,1.0,2016.0,ANTALYASPOR A.Ş.,AYTEMIZ ALANYASPOR,2 - 1,2.0,home


In [11]:
FifaData = pd.read_csv('C:/Users/v-ozgenc/Desktop/ratings16.csv')
FifaData.index = FifaData['Team']
cols = list(df)
cols[1], cols[0] = cols[0], cols[1]
df = df.ix[:,cols]
df = df.reset_index(drop=True)
df["homeATT"] = np.nan
df["homeMID"] = np.nan
df["homeDEF"] = np.nan
df["homeOVR"] = np.nan
df["awayATT"] = np.nan
df["awayMID"] = np.nan
df["awayDEF"] = np.nan
df["awayOVR"] = np.nan
fifateams = list(FifaData.Team)
teams = list(FullData.Team)
teams1 = list(df.home)
teams2 = list(df.away)
print(df.dtypes)
df

Season        float64
Difference    float64
away           object
home           object
score          object
week          float64
winner         object
homeATT       float64
homeMID       float64
homeDEF       float64
homeOVR       float64
awayATT       float64
awayMID       float64
awayDEF       float64
awayOVR       float64
dtype: object


c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


,Season,Difference,away,home,score,week,winner,homeATT,homeMID,homeDEF,homeOVR,awayATT,awayMID,awayDEF,awayOVR
0,2016.0,-1.0,BURSASPOR,ADANASPOR A.Ş.,1 - 2,1.0,away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016.0,0.0,ATIKER KONYASPOR,ÇAYKUR RIZESPOR A.Ş.,1 - 1,1.0,draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016.0,2.0,KASIMPAŞA A.Ş.,TRABZONSPOR A.Ş.,2 - 0,1.0,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016.0,3.0,ALANYASPOR,BEŞIKTAŞ A.Ş.,4 - 1,1.0,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016.0,-2.0,AKHISAR BELEDIYE GENÇLIK VE SPOR,KAYSERISPOR,0 - 2,1.0,away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016.0,2.0,GAZIANTEPSPOR,GENÇLERBIRLIĞI,2 - 0,1.0,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016.0,1.0,FENERBAHÇE A.Ş.,MEDIPOL BAŞAKŞEHIR FK,1 - 0,1.0,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2016.0,0.0,OSMANLISPOR FUTBOL KULÜBÜ,ANTALYASPOR A.Ş.,0 - 0,1.0,draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2016.0,1.0,KARDEMIR KARABÜKSPOR,GALATASARAY A.Ş.,1 - 0,1.0,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2016.0,1.0,ANTALYASPOR A.Ş.,AYTEMIZ ALANYASPOR,2 - 1,2.0,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
for x in teams:
    for y in teams1:
        if (y.find(x) != -1):
            df.ix[df.home == y, 'homeworth'] = FullData.at[x, 'TeamWorth']
            df.ix[df.home == y, 'homeage'] = FullData.at[x, 'TeamAge']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
for x in teams:
    for y in teams2:
        if (y.find(x) != -1):
            df.ix[df.away == y, 'awayworth'] = FullData.at[x, 'TeamWorth']
            df.ix[df.away == y, 'awayage'] = FullData.at[x, 'TeamAge']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
            
for x in teams:
    for y in teams1:
        if (y.find(x) != -1):
            df.ix[df.home == y, 'homesupport'] = FullData.at[x, 'Support']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
for x in teams:
    for y in teams2:
        if (y.find(x) != -1):
            df.ix[df.away == y, 'awaysupport'] = FullData.at[x, 'Support']

c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-d

In [13]:
spr = 'SPOR'
flag = False
for x in fifateams:
    for y in teams1:
        if(x.find(spr) != -1):
            flag = True
            #print(x)
            #print(flag)
            #print(x.find(spr), flag)
            x = x.replace(spr, '')
        if (y.find(x) != -1 and flag):
            #print(flag)
            x = x + spr
            flag = False
            #print(x, flag)
            df.ix[df.home == y, 'homeATT'] = FifaData.at[x, 'ATT']
            df.ix[df.home == y, 'homeMID'] = FifaData.at[x, 'MID']
            df.ix[df.home == y, 'homeDEF'] = FifaData.at[x, 'DEF']
            df.ix[df.home == y, 'homeOVR'] = FifaData.at[x, 'OVR']
        elif (y.find(x) != -1):
            #print('elif: ', flag)
            #print('elif: ', x, flag)
            df.ix[df.home == y, 'homeATT'] = FifaData.at[x, 'ATT']
            df.ix[df.home == y, 'homeMID'] = FifaData.at[x, 'MID']
            df.ix[df.home == y, 'homeDEF'] = FifaData.at[x, 'DEF']
            df.ix[df.home == y, 'homeOVR'] = FifaData.at[x, 'OVR']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
        if (flag):
            x = x + spr
        flag = False
for x in fifateams:
    for y in teams2:
        if(x.find(spr) != -1):
            flag = True
            #print(x)
            #print(flag)
            #print(x.find(spr), flag)
            x = x.replace(spr, '')
        if (y.find(x) != -1 and flag):
            #print(flag)
            x = x + spr
            flag = False
            #print(x, flag)
            df.ix[df.away == y, 'awayATT'] = FifaData.at[x, 'ATT']
            df.ix[df.away == y, 'awayMID'] = FifaData.at[x, 'MID']
            df.ix[df.away == y, 'awayDEF'] = FifaData.at[x, 'DEF']
            df.ix[df.away == y, 'awayOVR'] = FifaData.at[x, 'OVR']
        elif (y.find(x) != -1):
            #print('elif: ', flag)
            #print('elif: ', x, flag)
            df.ix[df.away == y, 'awayATT'] = FifaData.at[x, 'ATT']
            df.ix[df.away == y, 'awayMID'] = FifaData.at[x, 'MID']
            df.ix[df.away == y, 'awayDEF'] = FifaData.at[x, 'DEF']
            df.ix[df.away == y, 'awayOVR'] = FifaData.at[x, 'OVR']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
        if (flag):
            x = x + spr
        flag = False
df

c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\v-ozgenc\appdata\local\am

,Season,Difference,away,home,score,week,winner,homeATT,homeMID,homeDEF,...,awayATT,awayMID,awayDEF,awayOVR,homeworth,homeage,awayworth,awayage,homesupport,awaysupport
0,2016.0,-1.0,BURSASPOR,ADANASPOR A.Ş.,1 - 2,1.0,away,71.0,67.0,69.0,...,70.0,72.0,71.0,72.0,19.98,23.7,48.98,24.4,4594.0,16106.0
1,2016.0,0.0,ATIKER KONYASPOR,ÇAYKUR RIZESPOR A.Ş.,1 - 1,1.0,draw,76.0,68.0,68.0,...,72.0,70.0,71.0,71.0,34.68,22.6,28.75,25.3,3369.0,15723.0
2,2016.0,2.0,KASIMPAŞA A.Ş.,TRABZONSPOR A.Ş.,2 - 0,1.0,home,73.0,74.0,76.0,...,69.0,71.0,71.0,70.0,95.95,23.8,39.50,23.4,17252.0,1416.0
3,2016.0,3.0,ALANYASPOR,BEŞIKTAŞ A.Ş.,4 - 1,1.0,home,78.0,78.0,76.0,...,NaN,NaN,NaN,NaN,177.40,25.3,38.68,25.5,30448.0,4541.0
4,2016.0,-2.0,AKHISAR BELEDIYE GENÇLIK VE SPOR,KAYSERISPOR,0 - 2,1.0,away,73.0,72.0,70.0,...,70.0,71.0,71.0,70.0,45.85,25.0,50.20,26.7,5524.0,2501.0
5,2016.0,2.0,GAZIANTEPSPOR,GENÇLERBIRLIĞI,2 - 0,1.0,home,69.0,70.0,70.0,...,71.0,69.0,68.0,70.0,36.78,24.1,40.55,24.6,2625.0,4236.0
6,2016.0,1.0,FENERBAHÇE A.Ş.,MEDIPOL BAŞAKŞEHIR FK,1 - 0,1.0,home,78.0,75.0,73.0,...,80.0,77.0,78.0,77.0,72.95,26.4,168.68,24.8,3208.0,16485.0
7,2016.0,0.0,OSMANLISPOR FUTBOL KULÜBÜ,ANTALYASPOR A.Ş.,0 - 0,1.0,draw,81.0,71.0,72.0,...,74.0,74.0,72.0,73.0,41.80,25.4,56.70,25.1,12645.0,3792.0
8,2016.0,1.0,KARDEMIR KARABÜKSPOR,GALATASARAY A.Ş.,1 - 0,1.0,home,76.0,77.0,74.0,...,71.0,70.0,70.0,70.0,129.35,25.1,36.08,25.8,21351.0,4422.0
9,2016.0,1.0,ANTALYASPOR A.Ş.,AYTEMIZ ALANYASPOR,2 - 1,2.0,home,73.0,70.0,71.0,...,81.0,71.0,72.0,71.0,38.68,25.5,41.80,25.4,4541.0,12645.0


In [14]:
(df[df.winner == 'draw']['winner']) = 'Draw'
dRAW = lambda x: x.replace('draw', 'Draw')
df.ix[df.winner == 'draw', 'winner'] = (df[df.winner == 'draw']['winner']).apply(dRAW)

c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
df[df.winner != 'Draw'].count()/306

Season         0.781046
Difference     0.781046
away           0.781046
home           0.781046
score          0.781046
week           0.781046
winner         0.781046
homeATT        0.781046
homeMID        0.781046
homeDEF        0.781046
homeOVR        0.781046
awayATT        0.777778
awayMID        0.777778
awayDEF        0.777778
awayOVR        0.777778
homeworth      0.781046
homeage        0.781046
awayworth      0.781046
awayage        0.781046
homesupport    0.781046
awaysupport    0.781046
dtype: float64

In [16]:
df2 = pd.read_csv('C:/Users/v-ozgenc/Desktop/Out.csv', index_col = 'Unnamed: 0')
result = pd.concat([df, df2])
result = result.reset_index(drop = True)
result

,Season,Difference,away,home,score,week,winner,homeATT,homeMID,homeDEF,...,awayATT,awayMID,awayDEF,awayOVR,homeworth,homeage,awayworth,awayage,homesupport,awaysupport
0,2016.0,-1.0,BURSASPOR,ADANASPOR A.Ş.,1 - 2,1.0,away,71.0,67.0,69.0,...,70.0,72.0,71.0,72.0,19.98,23.7,48.98,24.4,4594.0,16106.0
1,2016.0,0.0,ATIKER KONYASPOR,ÇAYKUR RIZESPOR A.Ş.,1 - 1,1.0,Draw,76.0,68.0,68.0,...,72.0,70.0,71.0,71.0,34.68,22.6,28.75,25.3,3369.0,15723.0
2,2016.0,2.0,KASIMPAŞA A.Ş.,TRABZONSPOR A.Ş.,2 - 0,1.0,home,73.0,74.0,76.0,...,69.0,71.0,71.0,70.0,95.95,23.8,39.50,23.4,17252.0,1416.0
3,2016.0,3.0,ALANYASPOR,BEŞIKTAŞ A.Ş.,4 - 1,1.0,home,78.0,78.0,76.0,...,NaN,NaN,NaN,NaN,177.40,25.3,38.68,25.5,30448.0,4541.0
4,2016.0,-2.0,AKHISAR BELEDIYE GENÇLIK VE SPOR,KAYSERISPOR,0 - 2,1.0,away,73.0,72.0,70.0,...,70.0,71.0,71.0,70.0,45.85,25.0,50.20,26.7,5524.0,2501.0
5,2016.0,2.0,GAZIANTEPSPOR,GENÇLERBIRLIĞI,2 - 0,1.0,home,69.0,70.0,70.0,...,71.0,69.0,68.0,70.0,36.78,24.1,40.55,24.6,2625.0,4236.0
6,2016.0,1.0,FENERBAHÇE A.Ş.,MEDIPOL BAŞAKŞEHIR FK,1 - 0,1.0,home,78.0,75.0,73.0,...,80.0,77.0,78.0,77.0,72.95,26.4,168.68,24.8,3208.0,16485.0
7,2016.0,0.0,OSMANLISPOR FUTBOL KULÜBÜ,ANTALYASPOR A.Ş.,0 - 0,1.0,Draw,81.0,71.0,72.0,...,74.0,74.0,72.0,73.0,41.80,25.4,56.70,25.1,12645.0,3792.0
8,2016.0,1.0,KARDEMIR KARABÜKSPOR,GALATASARAY A.Ş.,1 - 0,1.0,home,76.0,77.0,74.0,...,71.0,70.0,70.0,70.0,129.35,25.1,36.08,25.8,21351.0,4422.0
9,2016.0,1.0,ANTALYASPOR A.Ş.,AYTEMIZ ALANYASPOR,2 - 1,2.0,home,73.0,70.0,71.0,...,81.0,71.0,72.0,71.0,38.68,25.5,41.80,25.4,4541.0,12645.0


In [17]:
result.to_csv('C:/Users/v-ozgenc/Desktop/Out.csv', encoding='utf-8')